# Image loading and generation notebook

## Notebook setup

In [ ]:
# noqa
import os
COLAB = 'DATALAB_DEBUG' in os.environ

if COLAB:
    !apt-get update
    !apt-get install git
    !git clone https://gist.github.com/oskopek/e27ca34cb2b813cae614520e8374e741 bstrap
    import bstrap.bootstrap as bootstrap
else:
    wd = %%pwd
    if wd.endswith('notebooks'):
        print('Current directory:', wd)
        %cd ..
        %pwd
    import resources.our_colab_utils.bootstrap as bootstrap

bootstrap.bootstrap(branch='master', packages='dotmap==1.2.20 keras==2.1.4 pydicom==1.0.2 Pillow==5.0.0')

if COLAB:
    !rm -rf bstrap

## Actual notebook

In [ ]:
# noqa
import csv
import os
from dotmap import DotMap

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import pydicom
import skimage.transform
import tensorflow as tf

import resources.data.loader as loader
import resources.image_utils as imutils
import resources.synthetic_data as synth_data

%load_ext autoreload
%autoreload 2
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.cmap'] = 'gray' # 'viridis', 'gray'

## Data setup

In [ ]:
breast_prefix = os.path.abspath('/home/oskopek/local/Breasts')
loader.init(breast_prefix)

## Define custom conversion and plotting

In [ ]:
# def convert(img, img_meta):
#     img = imutils.standardize(img, img_meta)
#     img = imutils.downsample(img)

#     img_norm = imutils.normalize_gaussian(img)
#     return img, img_norm


def show_img(img):
    f = plt.figure(figsize=(16, 8))
    ax = f.add_subplot(1, 2, 1)
    ax2 = f.add_subplot(1, 2, 2)
    ax.imshow(img)
    ax2.hist(np.ravel(img))
    plt.show()


## inBreast test

In [ ]:
# images, patients = loader.load_inbreast()
# filter_id = 'cc9e66c5b31baab8'

# for pid, p in patients.items():
#     if not pid.startswith(filter_id):
#         continue
#     print("PatientID:", pid, "#images:", len(p.image_metadata))
#     for i, img_meta in enumerate(p.image_metadata.values()):
#         print(i + 1, "\t", "Laterality:", img_meta.laterality, "View:", img_meta.view, "BiRads:", img_meta.birads,
#               "Cancer:", img_meta.cancer)
#         img = imutils.load_image(img_meta.image_path)
#         img_small, img_small_gaussian = convert(img, img_meta)
#         show_img(img_small_gaussian)


## bcdr test

In [ ]:
# images, patients = loader.load_bcdr('BCDR-D01')  # 'BCDR-D02', 'BCDR-DN01'
# filter_id = 3

# for pid, p in patients.items():
#     if pid != filter_id:
#         continue
#     print("PatientID:", pid, "#images:", len(p.image_metadata))
#     for i, img_meta in enumerate(p.image_metadata.values()):
#         print(i + 1, "\t", "Laterality:", img_meta.laterality, "View:", img_meta.view, "Age:", img_meta.age, "Cancer:",
#               img_meta.cancer)
#         img = imutils.load_image(img_meta.image_path)
#         img_small, img_small_gaussian = convert(img, img_meta)
#         show_img(img_small_gaussian)


## Dataset summary stats

In [ ]:
# def print_info(patients):
#     print("Patients:", len(patients))
#     print("Images:", sum([len(p.image_metadata.values()) for p in patients.values()]))

#     def f(p):
#         return [1 if i.cancer else 0 for i in p.image_metadata.values()]

#     cancer = [sum(f(p)) for p in patients.values()]
#     print("Cancer:", sum(cancer))

#     pcancer = 0
#     pocancer = 0
#     phealthy = 0
#     pohealthy = 0
#     for pid, p in patients.items():
#         hcnt = 0
#         ccnt = 0
#         for img_meta in p.image_metadata.values():
#             if img_meta.cancer:
#                 ccnt += 1
#             else:
#                 hcnt += 1
#         if hcnt > 0:
#             phealthy += 1
#         if ccnt > 0:
#             pcancer += 1
#         if hcnt == 0:
#             assert ccnt > 0
#             pocancer += 1
#         if ccnt == 0:
#             assert hcnt > 0
#             pohealthy += 1
#     print('pCancer:', pcancer, 'pHealthy:', phealthy, 'pOnlyCancer:', pocancer, 'pOnlyHealthy:', pohealthy, 'pTotal:',
#           len(patients))


In [ ]:
# print("Inbreast")
# _, patients_inb = loader.load_inbreast()
# print_info(patients_inb)
# print()

# print("BCDR-D01")
# _, patients_d01 = loader.load_bcdr('BCDR-D01')
# print_info(patients_d01)
# print()

# print("BCDR-D02")
# _, patients_d02 = loader.load_bcdr('BCDR-D02')
# print_info(patients_d02)
# print()

# print("BCDR-DN01")
# _, patients_dn01 = loader.load_bcdr('BCDR-DN01')
# print_info(patients_dn01)
# print()

# size = (800, 800)
# print("Gigabytes for all images in total with size {}: {}".format(size, (410 + 260 + 704 + 200) *
#                                                                   (size[0] * size[1]) * 8 / 1024**3))


## Convert datasets to trainable CycleGan Images

* Add labels to BCDR images
* Method to filter out CC view and resize and split according to label bcdr
* Add labels to inBreast images
* Method to filter out CC view and resize and split according to label inBreast
* Merge them and copy the images to 2 folders based on label

In [ ]:
def filter_view(images):
    res_healthy = []
    res_cancer = []
    for i, image in images.items():
        if image.view == 'CC':
            if image.cancer:
                res_cancer.append(image)
            else:
                res_healthy.append(image)
    return res_healthy, res_cancer


print("Inbreast")
images_inb, _ = loader.load_inbreast()
inb_healthy, inb_cancer = filter_view(images_inb)
print("Healthy:", len(inb_healthy), "Cancer:", len(inb_cancer))
print()

print("BCDR-D01")
images_d01, _ = loader.load_bcdr('BCDR-D01')
d01_healthy, d01_cancer = filter_view(images_d01)
print("Healthy:", len(d01_healthy), "Cancer:", len(d01_cancer))
print()

print("BCDR-D02")
images_d02, _ = loader.load_bcdr('BCDR-D02')
d02_healthy, d02_cancer = filter_view(images_d02)
print("Healthy:", len(d02_healthy), "Cancer:", len(d02_cancer))
print()

print("Overall")
healthy = inb_healthy + d01_healthy + d02_healthy
cancer = inb_cancer + d01_cancer + d02_cancer
print("Healthy:", len(healthy), "Cancer:", len(cancer))


In [ ]:
from multiprocessing import Pool as ThreadPool
import imgaug
from imgaug import augmenters as iaa
from imgaug import parameters as iap
from itertools import repeat

SEED = 42

imgaug.seed(SEED)
aug = iaa.Sequential([iaa.Affine(rotate=(-4, 4)), iaa.Affine(scale={"x": (0.98, 1.13), "y": (0.98, 1.13)})])
aug_img = iaa.ContrastNormalization((0.08, 1.2), per_channel=False)


def transform_img(img, mask, img_meta, augment=False, size=(256, 256)):
    img = imutils.standardize(img, img_meta)
    mask = imutils.standardize(mask, img_meta)
    img = imutils.downsample(img, size=size)
    mask = imutils.downsample(mask, size=size)

    if augment:
        aug_det = aug.to_deterministic()
        img = imutils.normalize_gaussian(img)
        img = imutils.normalize(img, new_min=0, new_max=255)
        mask = imutils.normalize_gaussian(mask)
        mask = imutils.normalize(mask, new_min=0, new_max=255)
        img, mask = aug_det.augment_images([img, mask])
        img = aug_img.augment_image(img)

    img = imutils.normalize_gaussian(img)
    img = imutils.normalize(img, new_min=0, new_max=255)
    mask = imutils.normalize_gaussian(mask)
    mask = imutils.normalize(mask, new_min=0, new_max=255)
    return img, mask


def to_feature(img):
    img_to_bytes = tf.compat.as_bytes(img.astype(np.float32).tostring())
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_to_bytes]))


def to_example(img, mask, label):
    assert img.shape == mask.shape
    assert isinstance(label, int)
    feature = {
            'image': to_feature(img),
            'mask': to_feature(mask),
            'width': tf.train.Feature(int64_list=tf.train.Int64List(value=[min_w])),
            'height': tf.train.Feature(int64_list=tf.train.Int64List(value=[min_h])),
            'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label])),
    }
    # Create an example protocol buffer
    example = tf.train.Example(features=tf.train.Features(feature=feature))
    return example


def f(fname_base, run_id, augment, label, size, inp):
    lst, thread_id = inp
    fname = "{}.r_{}.t_{}.tfrecord".format(fname_base, run_id, thread_id)
    imgaug.seed(SEED * run_id)
    options = tf.python_io.TFRecordOptions(compression_type=tf.python_io.TFRecordCompressionType.GZIP)
    with tf.python_io.TFRecordWriter(fname, options=options) as writer:
        for i, img_meta in lst:
            try:
                img = imutils.load_image(img_meta.image_path)
            except:
                print("Failed to load image", img_meta.image_path)
                continue
            try:
                mask = imutils.load_image(img_meta.mask_path)
            except:
                print("Failed to load mask", img_meta.mask_path)
                continue
            img, mask = transform_img(img, mask, img_meta, augment=augment, size=size)
            to_example(img, mask, label)


def transform(lst, folder, run_id, augment, label, size):
    THREADS = 8
    batch_size = len(lst) // THREADS + 1
    lst = list(enumerate(lst))
    lst = [lst[i:i + batch_size] for i in range(0, len(lst), batch_size)]
    lst = list(zip(lst, repeat(fname), repeat(run_id), repeat(augment), repeat(label), repeat(size)))
    print("Transforming (run_id={})".format(run_id))
    pool = ThreadPool(THREADS)
    fname_base = os.path.join(folder, "label_{}.augment_{}".format(label, augment))
    functools.partial(f, fname_base=fname_base, run_id=run_id, augment=augment, label=label, size=size)
    results = pool.map(f, lst)
    print("Transformed (run_id={})".format(run_id))


size = (256, 256)
transformed = os.path.join(breast_prefix, "small_all_{}x{}".format(size[0], size[1]))

# cancer and healthy should be lists of image_meta + fix masks
# add to image_meta whjich dataset it is
# shuffle image metas
# split iamge metas or tfrecords

augment_epochs = 1
for run_id in range(augment_epochs):
    transform(cancer, transformed, run_id, run_id != 0, label=1, size=size)

for run_id in range(augment_epochs):
    transform(healthy, transformed, run_id, run_id != 0, label=0, size=size)


In [ ]:
## TODO(oskopek): CBIS

### Merge shards

In [ ]:
def merge_shards(in_files, fname):
    options = tf.python_io.TFRecordOptions(compression_type=tf.python_io.TFRecordCompressionType.GZIP)
    for in_file in in_files:
        with tf.python_io.TFRecordWriter(fname, options=options) as writer:
            for record in tf.python_io.tf_record_iterator(in_file, options=options):
                writer.write(record)

In [ ]:
healthy = []
cancer = []
for file in os.listdir(transformed):
    path = os.path.join(transformed, file)
    if not file.endswith('tfrecord'):
        continue
    if file.startswith('label=1'):
        cancer.append(path)
    elif file.startswith('label=0'):
        healthy.append(path)
    else:
        raise ValueError('Invalid file: ' + str(path))

transformed_all = os.path.join(breast_prefix, "small_all_{}x{}_final".format(size[0], size[1]))
merge_shards(healthy, os.path.join(tranformed_all, 'healthy.tfrecord'))
merge_shards(cancer, os.path.join(tranformed_all, 'cancer.tfrecord'))

## Synthetic data

In [ ]:
data_gen = synth_data.generate_synth(size=(256, 256), max_thresh=2.5)
for i in range(5):
    img, mask, img_meta = next(data_gen)
    # Go from img to img+mask in the GAN
    show_img(img + mask)
    show_img(mask)
